In [1]:
import numpy as np
A = [[1, 1, 2, 2], [2, 1, 1, 2], [2, 2, 1, 1], [1, 2, 2, 1]]
B = np.array([[1, 1, 2, 2], [2, 2, 1, 1], [2, 2, 1, 1], [1, 2, 2, 1]], dtype=np.int8)
C = np.array([
            [0, 0, 0, 0], 
            [0, 1, 0, 1], 
            [0, 0, 2, 0], 
            [0, 1, 0, 0]], dtype=np.int8)
D = np.array([
            [1, 1, 0, 0], 
            [0, 1, 1, 0], 
            [0, 0, 1, 1], 
            [1, 0, 0, 1]], dtype=np.int8)
E = np.array([  # Unique 
            [1, 2, 2, 1], 
            [2, 1, 1, 2], 
            [2, 1, 0, 0], 
            [1, 2, 0, 0]], dtype=np.int8)

GRID = C.copy()
N = len(GRID)
N_2 = N // 2

## Solver

In [2]:
DIGITS = set(range(N))
FLIP = np.array([0, 2, 1])

def equal_count(grid, done: set):
    """Checks for equal count of 1s and 2s.
    param grid: 2D array
    param done: set of rows or cols that are already done
    Returns True if any changes are made."""
    changed = False

    for r in (DIGITS - done):
        row = grid[r]
        if np.sum(row == 1) == N // 2:
            grid[r, np.where(row == 0)[0]] = 2
            done.add(r)
            changed = True

        if np.sum(row == 2) == N // 2:
            grid[r, np.where(row == 0)[0]] = 1
            done.add(r)
            changed = True

    return changed

def subsequent(grid, done: set):
    """Checks for subsequent 1s and 2s in rows and cols.
    Returns True if any changes are made."""
    changed = False

    for r, row in enumerate(grid):
        for i in range(len(row)-1):
            if row[i] == row[i+1] != 0:
                if i < N-2 and not grid[r, i+2]: # if next is empty
                    grid[r, i+2] = FLIP[row[i]]
                    changed = True

                if i > 0 and not grid[r, i-1]: # if prev is empty
                    grid[r, i-1] = FLIP[row[i]]
                    changed = True

    if changed:
        done.update(np.where(np.all(grid != 0, axis=1))[0])

    return changed

def unique(grid, done: set):
    """Checks for unique rows and cols.
    Returns True if any changes are made."""
    changed = False

    # Find rows containing exacty two gaps
    gapped_rows_indx = np.where(np.sum(grid == 0, axis=1) == 2)[0]  # [2 3]
    candidates = grid[gapped_rows_indx]  # [[2 1 0 0], [1 0 0 2]]
    keks = grid[list(done)]  # [[2 1 1 2]]

    # Compare each candidate with other complete rows
    for i, candy in enumerate(candidates):
        # Find index of colored tiles
        idx = np.where(candy != 0)[0]  # [0 1]
        for kek in keks:
            if np.array_equal(kek[idx], candy[idx]):
                # Replace the two gaps with complementary colors
                gaps = np.where(candy == 0)[0]  # [2, 3]
                grid[gapped_rows_indx[i], gaps] = FLIP[kek[gaps]]

                done.add(gapped_rows_indx[i])
                changed = True
    
    return changed

def solve(grid):
    done_rows, done_cols = set(), set()
    while True:
        if equal_count(grid, done_rows):
            continue
        if equal_count(grid.T, done_cols):
            continue
        if subsequent(grid, done_rows):
            continue
        if subsequent(grid.T, done_cols):
            continue
        if unique(grid, done_rows):
            continue
        if unique(grid.T, done_cols):
            continue
        break
    return grid

solve(GRID)


array([[1, 2, 1, 2],
       [2, 1, 2, 1],
       [1, 2, 2, 1],
       [2, 1, 1, 2]], dtype=int8)

In [3]:
# SAVE AND LOAD configirations

# save = {str(i) : a for i, a in enumerate(configs)}  # Give them names 
configs = [[1, 0], A, B]  # Or just use a list
np.savez("duko_configs.npz", *configs)  # Save them

loaded = np.load("duko_configs.npz")
configs = loaded.values()
for kek in configs:
    print(kek)

[1 0]
[[1 1 2 2]
 [2 1 1 2]
 [2 2 1 1]
 [1 2 2 1]]
[[1 1 2 2]
 [2 2 1 1]
 [2 2 1 1]
 [1 2 2 1]]


## Generator

In [15]:
def pure_random():
    """Randomly fills the grid with 1s and 2s."""
    return np.random.randint(0, 3, size=(N, N))

kek = pure_random()
print(kek)    

[[2 0 1 2]
 [1 1 0 0]
 [1 0 1 1]
 [2 2 0 2]]


In [ ]:
DIGITS = set(range(N))
FLIP = np.array([0, 2, 1])

def equal_count(grid, done: set):
    """Checks for equal count of 1s and 2s.
    param grid: 2D array
    param done: set of rows or cols that are already done
    Returns True if any changes are made."""
    changed = False

    for r in (DIGITS - done):
        row = grid[r]
        if np.sum(row == 1) == N // 2:
            grid[r, np.where(row == 0)[0]] = 2
            done.add(r)
            changed = True

        if np.sum(row == 2) == N // 2:
            grid[r, np.where(row == 0)[0]] = 1
            done.add(r)
            changed = True

    return changed

def subsequent(grid, done: set):
    """Checks for subsequent 1s and 2s in rows and cols.
    Returns True if any changes are made."""
    changed = False

    for r, row in enumerate(grid):
        for i in range(len(row)-1):
            if row[i] == row[i+1] != 0:
                if i < N-2 and not grid[r, i+2]: # if next is empty
                    grid[r, i+2] = FLIP[row[i]]
                    changed = True

                if i > 0 and not grid[r, i-1]: # if prev is empty
                    grid[r, i-1] = FLIP[row[i]]
                    changed = True

    if changed:
        done.update(np.where(np.all(grid != 0, axis=1))[0])

    return changed

def unique(grid, done: set):
    """Checks for unique rows and cols.
    Returns True if any changes are made."""
    changed = False

    # Find rows containing exacty two gaps
    gapped_rows_indx = np.where(np.sum(grid == 0, axis=1) == 2)[0]  # [2 3]
    candidates = grid[gapped_rows_indx]  # [[2 1 0 0], [1 0 0 2]]
    keks = grid[list(done)]  # [[2 1 1 2]]

    # Compare each candidate with other complete rows
    for i, candy in enumerate(candidates):
        # Find index of colored tiles
        idx = np.where(candy != 0)[0]  # [0 1]
        for kek in keks:
            if np.array_equal(kek[idx], candy[idx]):
                # Replace the two gaps with complementary colors
                gaps = np.where(candy == 0)[0]  # [2, 3]
                grid[gapped_rows_indx[i], gaps] = FLIP[kek[gaps]]

                done.add(gapped_rows_indx[i])
                changed = True
    
    return changed



In [4]:
def valid_action(grid, pos, color):
    """Check that placing color at pos is valid."""
    r, c = pos
    if grid[r, c] != 0:  # Neccessary?
        return False

    # Equal count
    if np.sum(grid[r] == color) + 1 > N_2 or np.sum(grid[:, c] == color) + 1 > N_2:
        return False

    # Check gap subsequent
    if 0 < r < N-1 and grid[r-1, c] == grid[r+1, c] == color:
        return False
    if 0 < c < N-1 and grid[r, c-1] == grid[r, c+1] == color:
        return False

    # Check subsequent
    if r > 1 and grid[r-2, c] == grid[r-1, c] == color:
        return False
    if r < N-2 and grid[r+2, c] == grid[r+1, c] == color:
        return False
    if c > 1 and grid[r, c-2] == grid[r, c-1] == color:
        return False
    if c < N-2 and grid[r, c+2] == grid[r, c+1] == color:
        return False
    
    return True


In [5]:
def get_consequence_action(grid, full_rows: set, full_cols: set):
    """Returns first action that follows from current grid 
    or None if no such action exists (exhausted)."""
    if a := equal_count(grid, full_rows):
        return a
    if a := equal_count(grid.T, full_cols):
        return a
    if a := subsequent(grid, full_rows):
        return a
    if a := subsequent(grid.T, full_cols):
        return a
    if a := unique(grid, full_rows):
        return a
    if a := unique(grid.T, full_cols):
        return a
        
    return None


def explore_action(grid, pos, color, full_rows: set, full_cols: set):
    """Returns exhausted grid or None if invalid"""
    grid = grid.copy()    
    full_rows, full_cols = full_rows.copy(), full_cols.copy()
    new_action = (pos, color)

    # Try new action until grid is exhausted or it is invalid
    while valid_action(grid, *new_action):
        grid[pos] = color
        new_action = get_consequence_action(grid, full_rows, full_cols)
        if new_action is None:  # Exhausted
            return grid
        
    return None  # Invalid

def generate_grid():
    new = np.zeros((N, N), dtype=np.int8)
    free_tiles = [(i, j) for i in range(N) for j in range(N)]
    full_rows, full_cols = set(), set()

    while len(free_tiles) > 0:
        indx = np.random.randint(len(free_tiles))
        pos = free_tiles.pop(indx)
        if new[pos] != 0:
            continue

        color = np.random.randint(1, 3)
        result = explore_action(new, pos, color, full_rows, full_cols)
        if result is not None:
            new = result
        else:
            new[pos] = FLIP[color]
    
    return new


print(generate_grid())

[[1 2 1 1]
 [2 1 2 1]
 [2 2 1 2]
 [1 1 2 2]]


In [16]:
def three(full):
    full.add(1)

def two(full_rows, full_cols):
    three(full_rows)
    three(full_cols)

def one():
    full_rows, full_cols = set(), set()
    two(full_rows, full_cols)
    print(full_rows, full_cols)

one()

{1} {1}
